In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', _URL, extract=True)

In [ ]:
path_to_zip

In [ ]:
os.path.dirname(path_to_zip)

In [ ]:
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')
PATH

In [ ]:
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

In [ ]:
BATCH_SIZE = 32
IMG_SIZE = (160, 160)

In [ ]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(train_dir, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE)

In [ ]:
validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(validation_dir, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE)

In [ ]:
class_names = train_dataset.class_names
class_names

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype('uint8'))
        plt.title(class_names[labels[i]])

In [ ]:
val_batches = tf.data.experimental.cardinality(validation_dataset)

In [ ]:
val_batches

In [ ]:
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)

In [ ]:
print("Number of validation batches: {}".format(tf.data.experimental.cardinality(validation_dataset)))
print("Number of test batches: {}".format(tf.data.experimental.cardinality(test_dataset)))

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(AUTOTUNE)
validation_dataset = validation_dataset.prefetch(AUTOTUNE)
test_dataset = test_dataset.prefetch(AUTOTUNE)

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
for images, _ in train_dataset.take(1):
    plt.figure(figsize=(10, 10))
    first_img = images[0]
    
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_img = data_augmentation(tf.expand_dims(first_img, 0))
        plt.imshow(augmented_img[0] / 255)

In [ ]:
# 两种把[0, 255]缩放到[-1, 1]的方法，主要用于适配mobile_net的输入

# 方法1
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

# 方法2
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, -1)

In [ ]:
IMG_SIZE

In [ ]:
IMG_SIZE + (3,)

In [ ]:
IMG_SHAPE = IMG_SIZE + (3,)

In [ ]:
base_model = tf.keras.applications.MobileNetV2(IMG_SHAPE, include_top = False, weights='imagenet')

In [ ]:
image_batch, label_batch = next(iter(train_dataset))

feature_batch = base_model(image_batch)
feature_batch.shape

In [ ]:
base_model.trainable = False

In [ ]:
base_model.summary()

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
feature_batch_average.shape

In [ ]:
prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
prediction_batch.shape

In [ ]:
inputs = tf.keras.Input((160, 160, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)

model = tf.keras.Model(inputs, outputs)

In [ ]:
model.summary()

In [ ]:
base_learning_rate = 0.0001

model.compile(tf.keras.optimizers.Adam(base_learning_rate), loss = tf.keras.losses.BinaryCrossentropy(True), metrics=['accuracy'])

In [ ]:
len(model.trainable_variables)

In [ ]:
len(model.trainable_weights)

In [ ]:
loss0, acc0 = model.evaluate(validation_dataset)

In [ ]:
print("init loss: {:.3f}".format(loss0))
print("init acc: {:.3f}".format(acc0))

In [ ]:
initial_epochs = 10

In [ ]:
history = model.fit(train_dataset, epochs = initial_epochs, validation_data=validation_dataset)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
plt.figure(figsize=(8,8))

plt.subplot(2, 1, 1)
plt.plot(acc, label = 'Training Accuracy')
plt.plot(val_acc, label = 'Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()), 1])
plt.title('Training and validation accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label = 'Training Loss')
plt.plot(val_loss, label = 'Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Loss')
plt.ylim([min(plt.ylim()), 1])
plt.title('Training and validation loss')

plt.xlabel('Epoch')
plt.show()

In [ ]:
base_model.trainable = True

In [ ]:
print("Number of layers in base model: {}".format(len(base_model.layers)))

In [ ]:
fine_tune_at = 100

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.RMSprop(base_learning_rate / 10), loss=tf.keras.losses.BinaryCrossentropy(True), metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
len(model.trainable_variables)

In [ ]:
fine_tune_epochs = 10
total_epochs = initial_epochs + fine_tune_epochs

history_fine = model.fit(train_dataset, epochs=total_epochs, initial_epoch=history.epoch[-1], validation_data=validation_dataset)

In [ ]:
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']
loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))

plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Acc')
plt.plot(val_acc, label='Val Acc')
plt.ylim([0.8, 1])
plt.plot([initial_epochs - 1, initial_epochs - 1], plt.ylim(), label='Start fine tune')
plt.legend(loc='lower right')
plt.title('Training and val acc')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Val Loss')
plt.ylim([0, 1])
plt.plot([initial_epochs - 1, initial_epochs - 1], plt.ylim(), label='Start fine tune')
plt.legend(loc='upper right')
plt.title('Training and val Loss')

plt.xlabel('Epochs')
plt.show()

In [ ]:
loss, acc = model.evaluate(test_dataset)
print("Acc: {:.2f}".format(acc))

In [ ]:
image_batch, label_batch = test_dataset.as_numpy_iterator().next()

In [ ]:
image_batch.shape

In [ ]:
label_batch

In [ ]:
predictions = model.predict(image_batch)

In [ ]:
predictions

In [ ]:
predictions = predictions.flatten()

In [ ]:
predictions

In [ ]:
predictions = tf.nn.sigmoid(predictions)

In [ ]:
predictions

In [ ]:
predictions = tf.where(predictions < 0.5, 0, 1)

In [ ]:
predictions

In [ ]:
print("Prediction: {}".format(predictions.numpy()))
print("Label:      {}".format(label_batch))

In [ ]:
plt.figure(figsize=(10, 10))

for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image_batch[i].astype('uint8'))
    plt.title(class_names[predictions[i]])